This code (will be) set up to generate a forecasted impact map of a specified area.
The area in the example case is Houston, TX
The inundation extent is sourced from the National Water Model forecast. 
Depth model is calculated from the inundation extent and the national 3m DEM
The impact map is calculated from the depth and the CDC SVI scores at the census tract scale. 

In [ ]:
# edit code so that it runs automatically every hour
# use python.12.4 kernel
# THIS IS A WORK IN PROGRESS

Currently, the dependencies require numpy <2, so make sure your numpy package matches that requirement. If it does, skip the next two steps

In [14]:
# currently we need numpy <2. So downgrade numpy to 1.19.5
%pip uninstall numpy

^C
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install 'numpy<2'

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


## Import packages and set the paths to the working directory and input files
TO DO: 
1. Replace the directory path with the path to the folder you want to work in
2. Replace dem_name with path to the dem you want to use (as a tif) 
3. Replace the inund_polygon with path to the area extent you want to map (as a .shp)

In [21]:
# Import packages
import os
import arcpy
from datetime import datetime

# Format the current datetime as a string in the desired format (e.g., YYYYMMDD_HHMMSS)
datetime = datetime.now().strftime("%Y%m%d_%H%M%S")


# Replace 'path/to/your/directory' with the path to your desired working directory
directory = 'C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/'

# Create a new folder within the current working directory
resultfolder = "5dayforecast"+datetime
os.makedirs(os.path.join(directory, resultfolder), exist_ok=True)
print(f"New folder created at: {os.path.join(directory, resultfolder)}")
infolder = directory+resultfolder+'/'


# set up input variables
dem_name = 'C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/Houston_DEM.tif'
inund_polygon = 'C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/clipped_extent.shp'


# Set the working directory
os.chdir(infolder)
# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

# set up input and output variables
arcpy.env.overwriteOutput = True

# make a geodatabase within the infolder   
gdb_path = infolder 
# Create the geodatabase
arcpy.CreateFileGDB_management(out_folder_path = infolder, out_name='geodatabase.gdb')
#workspace
ws = arcpy.env.workspace = infolder

New folder created at: C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/5dayforecast20240705_093007
Current Working Directory:  C:\Users\Lyn\Documents\Programming\Python_codes\fwdet\Forecast_test_5day\5dayforecast20240705_093007



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found

## Query the NWM 5 day inundation exent. This section will download it as a shapefile, merge polygons, and prepare for depth calculaiton

In [3]:
# query the NOAA API for the most recent flood predictions
import requests
import geopandas as gpd

# Construct the query URL
# To change the query, you can go to https://maps.water.noaa.gov/server/rest/services/nwm
# Then select the link you want, copy it here, and add /0/query to the end
# e.g. https://link_to_service/0/query
query_url = "https://maps.water.noaa.gov/server/rest/services/nwm/mrf_gfs_5day_max_inundation_extent/MapServer/0/query"
params = {
    'where': '1=1',
    'outFields': '*',
    'outSR': '4326',  # Specify output spatial reference if needed
    'f': 'geojson',  # Request geojson output
    'returnGeometry': 'true'
}
# Make the request
response = requests.get(query_url, params=params)


# Check if the server is running correctly. A response of 500 means there is a server error. 
if response.status_code == 200:
    try:
        # Attempt to convert the response to a GeoDataFrame
        predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
        print(predictions.head())
    except ValueError as e:
        # Handle JSON decoding errors
        print(f"Error decoding JSON: {e}")
else:
    # The request failed; print the status code and response text
    print(f"Request failed with status {response.status_code}: {response.text}")



# Convert to GeoDataFrame
predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
print(predictions.head())


                                            geometry feature_id  \
0  POLYGON ((-96.70638 32.29966, -96.70670 32.299...    1446524   
1  MULTIPOLYGON (((-98.91162 27.26558, -98.91162 ...     325213   
2  MULTIPOLYGON (((-98.92063 27.22029, -98.92062 ...     325213   
3  MULTIPOLYGON (((-98.92538 27.20926, -98.92538 ...     325213   
4  MULTIPOLYGON (((-98.91939 27.25465, -98.91949 ...     325213   

   streamflow_cfs           reference_time              update_time  oid  
0          410.71  2024-07-05 06:00:00 UTC  2024-07-05 12:34:25 UTC    1  
1            8.12  2024-07-05 06:00:00 UTC  2024-07-05 12:34:25 UTC    2  
2            8.12  2024-07-05 06:00:00 UTC  2024-07-05 12:34:25 UTC    3  
3            8.12  2024-07-05 06:00:00 UTC  2024-07-05 12:34:25 UTC    4  
4            8.12  2024-07-05 06:00:00 UTC  2024-07-05 12:34:25 UTC    5  
                                            geometry feature_id  \
0  POLYGON ((-96.70638 32.29966, -96.70670 32.299...    1446524   
1  MULTIPOLYG

In [11]:
%pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.0-cp39-cp39-win_amd64.whl (16.5 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Lyn\\AppData\\Roaming\\Python\\Python39\\site-packages\\numpy\\linalg\\_umath_linalg.cp39-win_amd64.pyd'
Check the permissions.



clip the predictions to the dem extent

In [22]:
# clip the predictions to the extent of interest - defined by the dem_name above

import geopandas as gpd
import rasterio
from shapely.geometry import box


# Open the DEM file with rasterio
with rasterio.open(dem_name) as dem:
    # Read the entire file into a numpy array
    bounds = dem.bounds


# Create a bounding box polygon from the DEM bounds
bounding_box_polygon = box(*bounds)

# Clip the predictions GeoDataFrame to the bounding box
clipped_predictions = gpd.clip(predictions, bounding_box_polygon)
predictions = clipped_predictions



In [23]:
# export predictions as a shapefile

from datetime import datetime
# Format the current datetime as a string in the desired format (e.g., YYYYMMDD_HHMMSS)
datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

print(datetime)

# Export GeoDataFrame as a Shapefile
file_name = infolder+"/5day_inundationextent_{datetime}.shp"
predictions.to_file(file_name)

20240705_093032


C:\Users\Lyn\AppData\Local\Temp\ipykernel_16568\1087914008.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  predictions.to_file(file_name)


In [24]:
# If predictions is empty, print a message and exit the script
import sys
if predictions.empty:
    print("No flood predictions in this area found.")
    sys.exit(0)

## FWDET2.1 calculates depth map from inundation extent
### Written by Dr.Sagy Cohen

In [15]:
# use kernel arcgispro-py3 (python 3.9.16)
import arcpy

In [16]:

#clip_dem = '' #[Optional] - If empty, the clip_dem will be calculated with the Clip_management function
#cost_raster ='' #[Optional] - If empty, the CostRaster will be calculated below
WaterDepthOutput = 'WaterDepth_i10_s0p5'
iterations = 10
SlopeFiltering = True
SlopeThreshold = 0.5

In [25]:
# clean and create necessary variables
dem = arcpy.Raster(dem_name)
cell_size = dem.meanCellHeight
# Proper string representation of dem extent to be accepted by Clip_management method
extent = '{} {} {} {}'.format(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)

# Dont run these if cost_raster and clip_dem are already defined above
cost_raster = (((dem <= 0)*999)+1)
cost_raster.save('CostRaster.tif')

clip_dem = 'ClipDEM'

arcpy.management.Clip(dem, extent, clip_dem, inund_polygon, nodata_value= "-9999", clipping_geometry="ClippingGeometry", maintain_clipping_extent="NO_MAINTAIN_EXTENT")
clip_dem_ras = arcpy.Raster(clip_dem)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

...

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Lyn\AppData\Local\ESRI\conda\envs\arcgispro-p

AttributeError: _ARRAY_API not found

In [ ]:
# debugging - delete later
# check if shapefile has FID field
import arcpy

# Set the path to your polyline feature class
polyline_feature_class = infolder+'polyline.shp'

# List all fields in the feature class
fields = arcpy.ListFields(polyline_feature_class)

# Check if OBJECTID field exists
objectid_exists = any(field.name == "FID" for field in fields)

if not objectid_exists:
    print("FID field does not exist. Consider creating a new feature class.")
    # Here you can add code to create a new feature class or handle the issue as needed.
else:
    print("FID field exists.")

In [ ]:
# Calculate Boundary Raster
print('Focal iteration '+str(i+1))arcpy.env.overwriteOutput = True
polyline = infolder + '/polyline.shp'
arcpy.PolygonToLine_management(inund_polygon, polyline)
# Convert polyline to raster
with arcpy.EnvManager(snapRaster=clip_dem_ras):
    arcpy.conversion.PolylineToRaster(polyline, 'FID', 'linerast15', "MAXIMUM_LENGTH", "NONE", cell_size)
raster_polyline = arcpy.Raster('linerast15')
raster_polyline.save("rstr_poly")
# The input whose values will be used as the output cell values if the condition is false.
inFalseConstant = '#'
where_clause = 'VALUE >= 0'
#Extract the boundary cells elevation from DEM
boundary = arcpy.sa.Con(raster_polyline, dem, inFalseConstant, where_clause)
# boundary.save('boundary1')
#Smooth boundary raster
for i in range(iterations):
    OutRasTemp = arcpy.sa.FocalStatistics(boundary, "Rectangle 5 5 CELL", 'MEAN', 'DATA')
    boundary = arcpy.sa.Con(raster_polyline, OutRasTemp, inFalseConstant, where_clause)
    boundary.save('boundary'+str(i+1))
#Identify and remove ocean boundary cells
OutRasTemp = arcpy.sa.FocalStatistics(dem, 'Circle 2 CELL', 'MINIMUM', 'DATA') 
whereClause2 = 'VALUE > 0'
boundary = arcpy.sa.Con(OutRasTemp, boundary, inFalseConstant, whereClause2)
#boundary.save("boundaryAfterOcean")
if SlopeFiltering:
#calculate topo slope
    print('Calculating Slope')
    extent_clip = '{} {} {} {}'.format(boundary.extent.XMin, boundary.extent.YMin, boundary.extent.XMax, boundary.extent.YMax)
    with arcpy.EnvManager(extent=extent_clip):
        out_slope = arcpy.sa.Slope(dem, "PERCENT_RISE", 1, "GEODESIC", "METER")
        out_slope.save("Slope_m")
#Remove erroneous boundary cells 
    whereClause_slope = 'VALUE > ' + str(SlopeThreshold)
    boundary = arcpy.sa.Con(out_slope, boundary, inFalseConstant, whereClause_slope)
boundary.save("boundFinal")
print('Finished calculating the boundary raster')

In [ ]:
# Calculate water depth 

MULTIPLIER = 10000
boundary_int = arcpy.sa.Int(boundary * MULTIPLIER)
#boundary_int.save("boundary_int")
print('Running cost allocation')
with arcpy.EnvManager(snapRaster=None, extent="DEFAULT", mask=clip_dem):
    cost_alloc = arcpy.sa.CostAllocation(boundary_int, cost_raster, '#', '#', 'Value')

# Divide the result from the cost allocation function using the same constant used to create the integer
# representation of the boundary
cost_alloc = arcpy.sa.Float(cost_alloc) / MULTIPLIER
print('Cost Allocation raster generated')
print('Calculating estimated water depth')
water_depth = (cost_alloc - clip_dem_ras)
# Remove estimated water depths below 0 and change them to 0
water_depth = arcpy.sa.Con(water_depth > 0, water_depth,"#")

In [ ]:
# Run a low-pass filter
# ie smooth the water depth raster
print('Running low-pass Filter')
water_depth_filtered = arcpy.sa.Filter(water_depth, 'LOW', 'DATA')
waterDepthFilter2 = arcpy.sa.Con(clip_dem_ras, water_depth_filtered, '#', 'VALUE > 0')
#waterDepthFilter2.save(WaterDepthOutput+'_filtered')
print('Finished low-pass Filter calculation')
#waterDepthFilter2

Turn the shapefile of the depth map into a raster

In [ ]:
%pip install osgeo

In [ ]:
import os
import geopandas as gpd
from osgeo import gdal, ogr, osr

# Set the working directory
os.chdir(r'C:/Users/Lyn/Documents/Programming/Python_codes/Impact_Map_Forecasting/ results_5day_forecast')

# Read the shapefile using GeoPandas
#shapefile = gpd.read_file(r'D:/Hesham/CUAHSI/Geospatial/Jupyter N2/svi_data.shp')

# Open the shapefile using OGR
#input_shp = ogr.Open(r'D:/Hesham/CUAHSI/Geospatial/Jupyter N2/svi_data.shp')
#source_layer = input_shp.GetLayer()
#defn = source_layer.GetLayerDefn()

# Extract column names
column_names = []
for n in range(predictions.GetFieldCount()):
    fdefn = predictions.GetFieldDefn(n)
    column_names.append(fdefn.GetName())

# Define output raster location and pixel size
output_raster_location = r'D:/Hesham/CUAHSI/Geospatial/Jupyter N3/svi_raster.tif'
pixel_size = 0.0001

# Get extent of the source layer
xmin, xmax, ymin, ymax = predictions.GetExtent()

# Calculate resolution
x_res = int(round((xmax - xmin) / pixel_size))
y_res = int(round((ymax - ymin) / pixel_size))

# Create the target raster file
target_ds = gdal.GetDriverByName('GTiff').Create(output_raster_location, x_res, y_res, 1, gdal.GDT_Float32, ['COMPRESS=LZW'])
target_ds.SetGeoTransform((xmin, pixel_size, 0, ymax, 0, -pixel_size))

# Set projection
srs = osr.SpatialReference()
srs.ImportFromEPSG(4269)  # EPSG:4269 is NAD83
target_ds.SetProjection(srs.ExportToWkt())

# Get the raster band and set NoData value
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(-9999)
band.Fill(-9999)

# Rasterize the shapefile layer to the raster
gdal.RasterizeLayer(target_ds, [1], source_layer, options=['ALL_TOUCHED=TRUE', 'ATTRIBUTE=SVI'])

# Close the raster dataset
target_ds = None

print("Rasterization complete!")


## Run SVI using depth map